In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

"""li.sports_list 내부의 a 태그 정보 추출"""
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None
try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    # 페이지 접속
    url = "https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?idx=baseball"
    driver.get(url)
    logger.info("페이지 로드 완료")

    # 데이터 추출
    stadium_link_dic = {}
    stadium_list = []
    sports_list_items = driver.find_elements(By.CSS_SELECTOR, 'li.sports_list')
    logger.info(f"총 {len(sports_list_items)}개의 sports_list 항목 발견")

    for li_item in sports_list_items:
        try:
            # 각 li 내부의 span들에서 a 태그 찾기
            spans = li_item.find_elements(By.CSS_SELECTOR, 'span')
            for span in spans:
                try:
                    a_tag = span.find_element(By.CSS_SELECTOR, "a[href*='PouVSwRTRIN&idx=baseball']")
                    href = a_tag.get_attribute('href')
                    
                    try:
                        name = a_tag.text
                    except NoSuchElementException:
                        name = ""
                    stadium_list.append(name)
                    stadium_link_dic[name] = href

                    logger.debug(f"추출 완료: {name} - {href}")
                    
                except NoSuchElementException:
                    continue


        except Exception as e:
            logger.warning(f"항목 처리 중 오류: {str(e)}")
            continue

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")
finally:
    if driver:
        driver.quit()
        logger.info("드라이버 종료 완료")


print("\n[추출 결과]")
# stadium_list
stadium_link_dic

        

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:페이지 로드 완료
INFO:__main__:총 1개의 sports_list 항목 발견
INFO:__main__:드라이버 종료 완료



[추출 결과]


{'대구야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=1&area=27230510|143&rname=Xff3TxYgVPouVSwRTRIN&idx=baseball',
 '마산야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=2&area=48160720|155&rname=XRL/UeMkVPouVSwRTRIN&idx=baseball',
 '사직야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=3&area=26260590|159&rname=Xu0jUQj1VPouVSwRTRIN&idx=baseball',
 '목동야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=4&area=11470510|108&rname=XRUgUPANVPouVSwRTRIN&idx=baseball',
 '잠실야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=5&area=11710720|108&rname=XgkXUewYVPouVSwRTRIN&idx=baseball',
 '문학야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=6&area=28170740|112&rname=XRcvUvYNVPouVSwRTRIN&idx=baseball',
 '월명야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=7&area=45130680|140&rname=XgYLUAv5VPouVSwRTRIN&idx=baseball',
 '무등야구장': 'https://wwwnew.k

In [25]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

"""
경기장 이름을 입력받아 해당 페이지의 날씨 정보 추출
:param stadium_link_dic: {'경기장명': '링크'} 형식의 딕셔너리
:param stadium_name: 조회할 경기장 이름
:return: 테이블 헤더와 데이터를 포함한 딕셔너리
"""
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None

try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    all_rows = []

    for stadium_name in stadium_list:

        # 해당 경기장 페이지 접속
        driver.get(stadium_link_dic[stadium_name])
        logger.info(f"{stadium_name} 페이지 로드 시작")

        # 테이블 로딩 대기 (최대 15초)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table'))
            )
        except TimeoutException:
            logger.error("테이블 요소를 찾을 수 없음")
            

        # 테이블 데이터 추출
        table = driver.find_element(By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table')
        
        # thead에서 컬럼명 추출
        columns = [th.text for th in table.find_elements(By.CSS_SELECTOR, 'thead th')]
        
        # tbody에서 데이터 추출
        rows = []
        for tr in table.find_elements(By.CSS_SELECTOR, 'tbody tr'):
            row_data = [td.text for td in tr.find_elements(By.TAG_NAME, 'td')]
            if row_data:  # 빈 행 제외
                rows.append(row_data)

        # 결과 구조화
        result = {
            'stadium': stadium_name,
            'columns': columns,
            'rows': rows
        }

        all_rows.extend(rows)

        logger.info(f"{stadium_name} 데이터 추출 완료")
        print(f"{stadium_name} 데이터 추출 완료")

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")

all_rows = [[stadium_list[i]] + row for i, row in enumerate(all_rows)]
all_columns = ['stadium'] + columns

weather_stadium_df = pd.DataFrame(all_rows, columns=all_columns)

driver.quit()

weather_stadium_df.to_csv('weather_stadium_data.csv', index=False, encoding='utf-8-sig')

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:대구야구장 페이지 로드 시작
INFO:__main__:대구야구장 데이터 추출 완료


대구야구장 데이터 추출 완료


INFO:__main__:마산야구장 페이지 로드 시작
INFO:__main__:마산야구장 데이터 추출 완료


마산야구장 데이터 추출 완료


INFO:__main__:사직야구장 페이지 로드 시작
INFO:__main__:사직야구장 데이터 추출 완료


...

무등야구장 데이터 추출 완료


INFO:__main__:청주야구장 페이지 로드 시작
INFO:__main__:청주야구장 데이터 추출 완료


청주야구장 데이터 추출 완료


INFO:__main__:한밭야구장 페이지 로드 시작
INFO:__main__:한밭야구장 데이터 추출 완료


한밭야구장 데이터 추출 완료


In [43]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None

try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    all_rows = []

    for stadium_name in stadium_list:

        # 해당 경기장 페이지 접속
        driver.get(stadium_link_dic[stadium_name])
        logger.info(f"{stadium_name} 페이지 로드 시작")

        # 테이블 로딩 대기 (최대 15초)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'table.life_09 tbody tr'))
            )
        except TimeoutException:
            logger.error("테이블 요소를 찾을 수 없음")
            

        # 테이블 데이터 추출
        table = driver.find_element(By.CSS_SELECTOR, 'table.life_09 tbody tr td span')
        
        time_list = []
        rain_list = []
        temp_list = []
        for i in range(6):  # 필요한 시간대만 선택
            time_id = f"hour{i}time"
            time = driver.find_element(By.ID, time_id).text
            rain_id = f"hour{i}rainProb"
            rain_prob = driver.find_element(By.ID, rain_id).text
            temp_id = f"hour{i}temp"
            temp = driver.find_element(By.ID, temp_id).text
            time_list.append(f'{time}시')
            rain_list.append(rain_prob) 
            temp_list.append(temp)
            print(f"{time}시 강수 확률: {rain_prob}")
            print(f"{time}시 기온: {temp}")

        print(time_list)
        print(rain_list)
        print(temp_list)

        columns = ['stadium'] + time_list
        rows1 = [stadium_name] + rain_list
        rows2 = [stadium_name] + temp_list

        all_rows.append(rows1)
        all_rows.append(rows2)
        # rows = []
        # for tr in table.find_elements(By.CSS_SELECTOR, 'tbody tr'):
        #     row_data = [td.text for td in tr.find_elements(By.TAG_NAME, 'td')]
        #     rows.append(row_data)
        # # 결과 구조화
        # result = {
        #     'stadium': stadium_name,
        #     # 'columns': columns,
        #     'rows': rows
        # }

        # all_rows.extend(rows)

        logger.info(f"{stadium_name} 데이터 추출 완료")
        print(f"{stadium_name} 데이터 추출 완료")

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")

# all_rows = [[stadium_list[i]] + row for i, row in enumerate(all_rows)]
# all_columns = ['stadium'] + columns

weather_stadium_df = pd.DataFrame(all_rows, columns=columns)    

driver.quit()

weather_stadium_df

weather_stadium_df.to_csv('weather_stadium_data.csv', index=False, encoding='utf-8-sig')

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:대구야구장 페이지 로드 시작
INFO:__main__:대구야구장 데이터 추출 완료


-시 강수 확률: -
-시 기온: 22
14시 강수 확률: 30
14시 기온: 23
15시 강수 확률: 30
15시 기온: 23
16시 강수 확률: 30
16시 기온: 23
17시 강수 확률: 30
17시 기온: 23
18시 강수 확률: 30
18시 기온: 22
['-시', '14시', '15시', '16시', '17시', '18시']
['-', '30', '30', '30', '30', '30']
['22', '23', '23', '23', '23', '22']
대구야구장 데이터 추출 완료


INFO:__main__:마산야구장 페이지 로드 시작
INFO:__main__:마산야구장 데이터 추출 완료


-시 강수 확률: -
-시 기온: -
-시 강수 확률: -
-시 기온: -
-시 강수 확률: -
-시 기온: 19
16시 강수 확률: 30
16시 기온: 19
17시 강수 확률: 30
17시 기온: 19
18시 강수 확률: 30
18시 기온: 18
['-시', '-시', '-시', '16시', '17시', '18시']
['-', '-', '-', '30', '30', '30']
['-', '-', '19', '19', '19', '18']
마산야구장 데이터 추출 완료


INFO:__main__:사직야구장 페이지 로드 시작
INFO:__main__:사직야구장 데이터 추출 완료


...

-시 강수 확률: -
-시 기온: -
-시 강수 확률: 40
-시 기온: 25
15시 강수 확률: 40
15시 기온: 25
16시 강수 확률: 40
16시 기온: 25
17시 강수 확률: 40
17시 기온: 25
18시 강수 확률: 40
18시 기온: 23
['-시', '-시', '15시', '16시', '17시', '18시']
['-', '40', '40', '40', '40', '40']
['-', '25', '25', '25', '25', '23']
무등야구장 데이터 추출 완료


INFO:__main__:청주야구장 페이지 로드 시작
INFO:__main__:청주야구장 데이터 추출 완료


-시 강수 확률: 40
-시 기온: 24
14시 강수 확률: 40
14시 기온: 25
15시 강수 확률: 40
15시 기온: 25
16시 강수 확률: 40
16시 기온: 25
17시 강수 확률: 40
17시 기온: 26
18시 강수 확률: 40
18시 기온: 25
['-시', '14시', '15시', '16시', '17시', '18시']
['40', '40', '40', '40', '40', '40']
['24', '25', '25', '25', '26', '25']
청주야구장 데이터 추출 완료


INFO:__main__:한밭야구장 페이지 로드 시작
INFO:__main__:한밭야구장 데이터 추출 완료


-시 강수 확률: -
-시 기온: -
-시 강수 확률: -
-시 기온: 25
15시 강수 확률: 40
15시 기온: 25
16시 강수 확률: 40
16시 기온: 26
17시 강수 확률: 40
17시 기온: 26
18시 강수 확률: 40
18시 기온: 25
['-시', '-시', '15시', '16시', '17시', '18시']
['-', '-', '40', '40', '40', '40']
['-', '25', '25', '26', '26', '25']
한밭야구장 데이터 추출 완료
